# Nuclei Detect  demo

This notebook can be used to predict cells and nuclei given one has the sufficient model and data

In [1]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import random
#import requests
from io import BytesIO
from PIL import Image
import numpy as np
import os
import cv2
from matplotlib.image import imread


Those are the relevant imports for the detection model

In [2]:
from maskrcnn_benchmark.config import cfg
pylab.rcParams['figure.figsize'] = 20, 12

# importing the prediction class
from predictor import NUCLEIdemo

ImportError: libcudart.so.10.0: cannot open shared object file: No such file or directory

The NUCLEIdemo class can load the config file and does the image prediction.

In [5]:
configuration_file = "../configs/nuclei_1gpu_nonorm_offline_res50.yaml"
#configuration_file = "/home/max/github/nuclei_cell_detect/configs/nuclei_1gpu_nonorm_offline.yaml"
# update the config options with the config file
cfg.merge_from_file(configuration_file)

# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])

# change dimensions of test images
cfg.merge_from_list(['INPUT.MAX_SIZE_TEST','2049'])

# change number of classes
cfg.merge_from_list(['MODEL.ROI_BOX_HEAD.NUM_CLASSES','4'])

# change normalization, here model was not normalized
cfg.merge_from_list(['INPUT.PIXEL_MEAN', [0., 0., 0.]])

# define model to use here
cfg.merge_from_list(['MODEL.WEIGHT', '/data/proj/smFISH/Students/Max_Senftleben/files/models/20190310_offline_augment/model_final.pth'])
cfg.merge_from_list(['OUTPUT_DIR', '.'])

# define how many objects can be identified per image
cfg.merge_from_list(['TEST.DETECTIONS_PER_IMG', '200'])

# show the configuration
#print(cfg)

Change the confidence threshold.

In [6]:
# load image
def load(path):
    pil_image = Image.open(path).convert("RGB")
    #print(pil_image)
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_matplot(path):
    img = imread(path)
    return img

def load_cv2(path):
    img = cv2.imread(path,cv2.IMREAD_ANYDEPTH)
    
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img = cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
    img = np.uint8(img)
    #img = cv2.convertScaleAbs(img)
    return img

def load_pil(path):
    img = Image.open(path)
    image = np.array(img)
    
    info = np.iinfo(image.dtype) # Get the information of the incoming image type
    print(info)
    data = image.astype(np.int32) / info.max # normalize the data to 0 - 1
    data = 255 * data # Now scale by 255
    img = data.astype(np.uint8)
    cv2.imshow("Window", img)
    
'''
    >>> img = np.empty((100,100,1),dtype = np.uint16)
>>> image = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

>>> cvuint8 = cv2.convertScaleAbs(image)

>>> cvuint8.dtype
dtype('uint8')
    
'''

# show image alongside the result and save if necessary
def imshow(img, result, save_path=None):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(img)
    plt.axis('off')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(result)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches = 'tight')
        plt.show()
    else:
        plt.show()
        
def imshow_single(result, save_path=None):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.imshow(result)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

Let's define a few helper functions for loading images from a URL

In [ ]:
nuclei_detect = NUCLEIdemo(
    cfg,
    min_image_size=1024,
    confidence_threshold=0.00000001,
)

### Define the image paths and show the results

In [ ]:
img_path = '../../ms2/ssss/'
#img_path = '/data/proj/smFISH/Students/Max_Senftleben/files/data/20190309_aug_pop/ss/'
img_path = '/data/proj/smFISH/Simone/test_intron/AMEXP20181106/AMEXP20181106_hyb1/test_run_20181123_AMEXP20181106_hyb1_filtered_png/test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_png/'
img_path = '/fish/simone_processing/simone_data/max_dapi_bf/EXP-19-CX9100_Excitatory_Hybridization15_DAPI/EXP-19-CX9100_Excitatory_Hybridization15_DAPI_filtered_png/'

# random image is taken from the image path
random_img = random.choice(os.listdir(img_path))
image = load(img_path + random_img)
image_matplot = load_matplot(img_path + random_img)
image_cv2 = load_pil(img_path + random_img)

print(type(image_cv2))
print(image_cv2.shape)
print(image_cv2.dtype)

result, prediction = nuclei_detect.run_on_opencv_image(image_cv2)
imshow(image_cv2, result)

print(vars(prediction))


In [42]:
def make_numpy(prediction, image, path):
    list_masks = vars(prediction)['extra_fields']['mask']
    masks_to_save = []
    img = np.squeeze(np.dsplit(image,3)[0], axis=2)
    masks_to_save.append(img)
    # iterate through the list of masks
    for i, label in enumerate(vars(prediction)['extra_fields']['labels']):
        numpy_mask = list_masks[i].numpy().transpose(1,2,0)
        numpy_mask = np.squeeze(numpy_mask, axis=2)
        numpy_mask[numpy_mask > 0] = label
        
        masks_to_save.append(numpy_mask)
    
    np.save(path, np.dstack(masks_to_save))
    

In [25]:
# predict for a folder of images
img_path = '/data/proj/smFISH/Simone/test_intron/AMEXP20181106/AMEXP20181106_hyb1/test_run_20181123_AMEXP20181106_hyb1_filtered_png/test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_png/'
save_results = '/data/proj/smFISH/Students/Max_Senftleben/files/results/'
save_independently = save_results + '20190329_test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_png/'
save_npy = save_results + '20190329_test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_npy/'

# done on cpu to check if there is a difference in prediction
save_independently_cpu = '/data/proj/smFISH/Students/Max_Senftleben/files/results/20190331_AMEXP20181106_DAPI_filtered_predicted_with_cpu/'

def save_pred_as_numpy():
    
    for one_image in os.listdir(img_path):
        print("Image {} is handled.".format(one_image))
        image = load_cv2(img_path + one_image)

        # normalization ca be applied
        result, prediction = nuclei_detect.run_on_opencv_image(image)
        img = Image.fromarray(result)
        img.save(save_independently + one_image[:-4] + '_pred.png')
        make_numpy(prediction, image, save_npy + one_image[:-4] + '_pred.npy')
        #imshow(image, result)
        
#main()

In [26]:
#make_numpy(prediction, image_cv2, 'test.npy')

# check numpy files
save_npy = '/data/proj/smFISH/Students/Max_Senftleben/files/results/20190329_test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_npy/'
random_img = random.choice(os.listdir(save_npy))
mask = np.load(save_npy+random_img)
mask_list = np.dsplit(mask, mask.shape[2])
#for i in mask_list:
    #print(i)
    #plt.imshow(np.squeeze(i, axis=2))
    #plt.show()
    #print(np.unique(i))

In [193]:
def chunking_labeled_images(number_chunks_dimension, old_chunks, new_chunks):
    for i in os.listdir(old_chunks):
        mask = np.load(old_chunks + i)
        
        height, width = mask.shape[:2]
        instance_count = mask.shape[2]
        #masklist = np.dsplit(mask, instance_count)
        #plt.imshow(np.dstack((masklist[1]*100, masklist[1]*100, masklist[1]*100)))
        #plt.show()
        
        hsplits = np.split(mask,number_chunks_dimension,axis=0)
        total_images = []
        for split in hsplits:
                total_images.append(np.split(split,number_chunks_dimension,axis=1))
        total_images = [img for cpl in total_images for img in cpl] 
        
        for idx,image_chunk in enumerate(total_images):
            image_chunks_ids = []
            mask = image_chunk != 0
            planes_to_keep = np.flatnonzero((mask).sum(axis=(0,1)))
            # Make sure that the image has labeled objects
            if planes_to_keep.size:
                image_chunk_trimmed = image_chunk[:,:,planes_to_keep]
                image_chunk_trimmed_id = new_chunks + i.split('.')[0]+'chunk'+str(idx)
        

                np.save(image_chunk_trimmed_id, image_chunk_trimmed)
        
        
    '''
        mask = np.zeros([height, width, instance_count],
                        dtype=np.uint8)

        class_ids = []
        mask_layer = 0
        for lab, pgs in polygons.items():
            
            # Get the class id
            class_id = class_names.index(lab)

            for obj_num, coords in pgs.items():
                
                rr, cc = skimage.draw.polygon(coords[:,0], coords[:,1])
                mask[rr, cc, mask_layer] = class_id
                mask_layer += 1
                class_ids.append(class_id)


        class_ids = np.array(class_ids)

        # account for colored images
        if image.shape[2] > 1:
            print('ss')
            dstack = np.dsplit(image,3)
            stack1 = dstack[0][:, :, 0]
            stack2 = dstack[1][:, :, 0]
            stack3 = dstack[2][:, :, 0]
            image_mask = np.insert(mask, 0, stack1, axis=2)
            image_mask = np.insert(image_mask, 0, stack2, axis=2)
            image_mask = np.insert(image_mask, 0, stack3, axis=2)
        else:
            image_mask = np.insert(mask,0,image,axis=2)
        
        print(image_mask.shape)
        sstack = np.dsplit(image_mask, image_mask.shape[2])
        plt.imshow(np.dstack((sstack[2], sstack[1], sstack[0])))
        plt.show()
        
        # stack image with masks

        # consider that we are collecting square images and only even number of cuts
        hsplits = np.split(image_mask,number_chunks_dimension,axis=0)
        total_images = []
        for split in hsplits:
                total_images.append(np.split(split,number_chunks_dimension,axis=1))
        total_images = [img for cpl in total_images for img in cpl] 

        # Chunk the images and the segmented data
        for idx,image_chunk in enumerate(total_images):
            image_chunks_ids = []
            mask = image_chunk != 0
            planes_to_keep = np.flatnonzero((mask).sum(axis=(0,1)))
            # Make sure that the image has labeled objects
            if planes_to_keep.size:
                image_chunk_trimmed = image_chunk[:,:,planes_to_keep]
                image_chunk_trimmed_id = image_id.split('.')[0]+'chunk'+str(idx)

                np.save(chunked_dir + image_chunk_trimmed_id, image_chunk_trimmed)
    '''

In [194]:
old = '/data/proj/smFISH/Students/Max_Senftleben/files/results/20190329_test_run_20181123_AMEXP20181106_hyb1_DAPI_filtered_npy/'
new = '/data/proj/smFISH/Students/Max_Senftleben/files/data/20190422_AMEX_transfer_nuclei/npy/'
dim = 2

chunking_labeled_images(dim, old, new)
